In [155]:
import ib_insync
from ib_insync import *

import pickle
import pandas as pd
import time
from datetime import datetime, timedelta, timezone
import numpy as np
import os

In [156]:
TICKERS = ["SPY", "SH", "SDS", "UPRO", "SSO", "SPXU"]
LEVERAGES = {"SPY": "1x", "SSO": "2x", "UPRO": "3x", "SH":"-1x", "SDS":"-2x", "SPXU": "-3x"}

In [157]:
print(ib_insync.__all__)
util.startLoop()

ib = IB()
ib.connect('127.0.0.1', 4002, clientId=1)

['util', 'Event', 'SoftDollarTier', 'PriceIncrement', 'Execution', 'CommissionReport', 'BarDataList', 'RealTimeBarList', 'BarData', 'RealTimeBar', 'HistogramData', 'NewsProvider', 'DepthMktDataDescription', 'ScannerSubscription', 'ScanDataList', 'FundamentalRatios', 'ExecutionFilter', 'PnL', 'PnLSingle', 'AccountValue', 'TickData', 'TickByTickAllLast', 'TickByTickBidAsk', 'TickByTickMidPoint', 'HistoricalTick', 'HistoricalTickBidAsk', 'HistoricalTickLast', 'TickAttrib', 'TickAttribBidAsk', 'TickAttribLast', 'MktDepthData', 'DOMLevel', 'TradeLogEntry', 'FamilyCode', 'SmartComponent', 'PortfolioItem', 'Position', 'Fill', 'OptionComputation', 'OptionChain', 'Dividends', 'NewsArticle', 'HistoricalNews', 'NewsTick', 'NewsBulletin', 'ConnectionStats', 'Contract', 'Stock', 'Option', 'Future', 'ContFuture', 'Forex', 'Index', 'CFD', 'Commodity', 'Bond', 'FuturesOption', 'MutualFund', 'Warrant', 'Bag', 'TagValue', 'ComboLeg', 'DeltaNeutralContract', 'ContractDetails', 'ContractDescription', 'Sca

<IB connected to 127.0.0.1:4002 clientId=1>

In [4]:
contract1 = Stock('SPXU', 'SMART', 'USD')
ib.reqMktData(contract1, '', False, False)
t1 = ib.ticker(contract1)

Error 354, reqId 3: Requested market data is not subscribed.Delayed market data is available.Error&BEST/STK/Top&BEST/STK/Top, contract: Stock(symbol='SPXU', exchange='SMART', currency='USD')


In [44]:
print(t1.bid, t1.bidSize, t1.ask, t1.askSize)

18.65 361 18.66 201


In [48]:
print(t1.time.replace(tzinfo=timezone.utc).astimezone(tz=None))

2021-06-24 21:47:39.384887+08:00


In [55]:
contract2 = Stock('SDS', 'SMART', 'USD')
ib.reqMktData(contract2, '', False, False)
t2 = ib.ticker(contract2)

In [59]:
print(t2.bid, t2.bidSize, t2.ask, t2.askSize)

9.19 700 9.2 4828


In [66]:
del t1
del t2

In [158]:
ib.reqMarketDataType(3)

In [159]:
contract1 = Stock(TICKERS[0], 'SMART', 'USD')
ib.reqMktData(contract1, '', False, False)

contract2 = Stock(TICKERS[1], 'SMART', 'USD')
ib.reqMktData(contract2, '', False, False)

contract3 = Stock(TICKERS[2], 'SMART', 'USD')
ib.reqMktData(contract3, '', False, False)

contract4 = Stock(TICKERS[3], 'SMART', 'USD')
ib.reqMktData(contract4, '', False, False)

contract5 = Stock(TICKERS[4], 'SMART', 'USD')
ib.reqMktData(contract5, '', False, False)

contract6 = Stock(TICKERS[5], 'SMART', 'USD')
ib.reqMktData(contract6, '', False, False)

t1 = ib.ticker(contract1)
t2 = ib.ticker(contract2)
t3 = ib.ticker(contract3)
t4 = ib.ticker(contract4)
t5 = ib.ticker(contract5)
t6 = ib.ticker(contract6)

In [160]:
print(t1.bid)

433.34


In [161]:
def read_multiple_prices(tickers, minutes):
#     contracts = list()
#     ib_tickers = list()
    
#     for ticker in tickers:
#         contract = Stock(ticker, 'SMART', 'USD')
# #         contracts.append(contract)
#         ib.reqMktData(contract, '', False, False)
#         ib_tickers.append(ib.ticker(contract))
# #         print(ib.tickers[0]bid)
    
# #     print(tickers[0])

    ib.sleep(0.06)
#     print(t1)
#     print(t1.bid, t1.bidSize, t1.ask, t1.askSize)
#     print(t2.bid, t2.bidSize, t2.ask, t2.askSize)
    
    data = []
    ten_min_data = []
    pre_row_data = None
    
    t_start = time.time()
    prev_time, next_time = t_start, t_start + 60 * 10
    t_end = t_start + 60 * minutes
    print(datetime.now())
    
    while time.time() < t_end:
        if time.time() > next_time:
            with open("data/{}_{}.pkl".format(prev_time, next_time), 'wb') as f:
                pickle.dump(ten_min_data, f)
            prev_time, next_time = next_time, next_time + 60 * 10
            ten_min_data = []
            
        ib.sleep(0.05)
        
        row_data = []
#         print(t1.bid, t1.bidSize, t1.ask, t1.askSize)
        row_data.extend([t1.bid, t1.bidSize, t1.ask, t1.askSize])
        row_data.extend([t2.bid, t2.bidSize, t2.ask, t2.askSize])
        row_data.extend([t3.bid, t3.bidSize, t3.ask, t3.askSize])
        row_data.extend([t4.bid, t4.bidSize, t4.ask, t4.askSize])
        row_data.extend([t5.bid, t5.bidSize, t5.ask, t5.askSize])
        row_data.extend([t6.bid, t6.bidSize, t6.ask, t6.askSize])
        
        if row_data != pre_row_data:
            print(row_data)
            row_time = t1.time.replace(tzinfo=timezone.utc).astimezone(tz=None)
            row = [row_time] + row_data
            data.append(row)
            ten_min_data.append(row)
            pre_row_data = row_data
        else:
            pass
        
    columns=['time']
    for i in range(1, 7):
        columns.extend(['bid{}'.format(i), 'bid_size{}'.format(i), 'ask{}'.format(i), 'ask_size{}'.format(i)])
    
    return pd.DataFrame(data, columns=columns)

In [162]:
%time
df = read_multiple_prices(TICKERS, 450)
df.to_csv('six_asset_data_0707_1.csv')

CPU times: user 5 µs, sys: 1 µs, total: 6 µs
Wall time: 20.3 µs
2021-07-07 09:22:50.052374
[433.35, 39, 433.37, 4, 15.2, 3064, 15.21, 3459, 8.83, 2086, 8.84, 2414, 117.57, 1, 117.58, 10, 122.21, 2, 122.22, 6, 17.55, 123, 17.56, 292]
[433.35, 39, 433.37, 4, 15.2, 3064, 15.21, 3459, 8.83, 2086, 8.84, 2414, 117.57, 2, 117.59, 11, 122.21, 1, 122.22, 1, 17.55, 123, 17.56, 292]
[433.35, 19, 433.37, 4, 15.2, 3064, 15.21, 3459, 8.83, 2086, 8.84, 2414, 117.57, 2, 117.59, 11, 122.21, 1, 122.22, 1, 17.55, 123, 17.56, 292]
[433.35, 19, 433.37, 4, 15.2, 3064, 15.21, 3459, 8.83, 2086, 8.84, 2414, 117.57, 2, 117.59, 11, 122.21, 1, 122.22, 1, 17.55, 123, 17.55, 1]
[433.35, 19, 433.37, 4, 15.2, 3064, 15.21, 3459, 8.83, 2086, 8.84, 2414, 117.57, 2, 117.59, 11, 122.2, 37, 122.22, 1, 17.55, 123, 17.55, 1]
[433.36, 8, 433.39, 61, 15.2, 3064, 15.21, 3459, 8.83, 2086, 8.84, 2414, 117.56, 13, 117.59, 11, 122.2, 37, 122.22, 1, 17.55, 123, 17.55, 1]
[433.36, 8, 433.39, 61, 15.2, 3349, 15.21, 3459, 8.83, 2086, 8

[433.39, 20, 433.4, 5, 15.2, 2391, 15.21, 3459, 8.83, 1032, 8.83, 1, 117.59, 10, 117.61, 6, 122.22, 9, 122.24, 4, 17.54, 162, 17.55, 126]
[433.39, 20, 433.4, 5, 15.2, 2391, 15.21, 3459, 8.83, 1032, 8.83, 1, 117.59, 10, 117.61, 6, 122.22, 9, 122.24, 4, 17.54, 117, 17.55, 233]
[433.41, 7, 433.42, 1, 15.2, 2387, 15.21, 3459, 8.83, 1032, 8.83, 1, 117.6, 34, 117.61, 1, 122.24, 31, 122.25, 1, 17.54, 117, 17.55, 233]
[433.41, 7, 433.42, 1, 15.2, 2387, 15.21, 3459, 8.83, 329, 8.83, 124, 117.6, 34, 117.61, 1, 122.24, 31, 122.25, 1, 17.54, 117, 17.55, 233]
[433.41, 7, 433.42, 1, 15.2, 2387, 15.21, 3459, 8.83, 329, 8.83, 124, 117.6, 34, 117.61, 1, 122.23, 21, 122.25, 6, 17.54, 117, 17.55, 243]
[433.42, 3, 433.43, 5, 15.2, 2387, 15.21, 3459, 8.83, 329, 8.83, 124, 117.62, 6, 117.63, 2, 122.23, 21, 122.25, 6, 17.54, 117, 17.55, 243]
[433.42, 3, 433.43, 5, 15.2, 2387, 15.21, 3459, 8.83, 52, 8.83, 124, 117.62, 6, 117.63, 2, 122.23, 21, 122.25, 6, 17.54, 117, 17.55, 243]
[433.43, 2, 433.45, 183, 15.2, 

[433.35, 36, 433.37, 52, 15.2, 3507, 15.21, 4309, 8.83, 1, 8.83, 1356, 117.58, 1, 117.59, 22, 122.2, 62, 122.23, 5, 17.55, 82, 17.56, 314]
[433.35, 26, 433.37, 38, 15.2, 3507, 15.21, 4309, 8.83, 1, 8.83, 1356, 117.58, 1, 117.59, 22, 122.2, 62, 122.23, 5, 17.55, 82, 17.56, 314]
[433.32, 13, 433.35, 215, 15.2, 3543, 15.21, 4359, 8.83, 215, 8.83, 154, 117.54, 1, 117.56, 25, 122.19, 11, 122.22, 37, 17.55, 234, 17.56, 254]
[433.32, 13, 433.35, 215, 15.2, 3543, 15.21, 4359, 8.83, 215, 8.83, 154, 117.54, 1, 117.56, 25, 122.19, 10, 122.22, 41, 17.55, 234, 17.56, 254]
[433.32, 1, 433.33, 7, 15.2, 3543, 15.21, 4359, 8.83, 215, 8.83, 154, 117.53, 4, 117.55, 5, 122.19, 10, 122.22, 41, 17.55, 234, 17.56, 254]
[433.32, 1, 433.33, 7, 15.2, 3543, 15.21, 4359, 8.83, 215, 8.83, 139, 117.53, 4, 117.55, 5, 122.2, 20, 122.22, 4, 17.55, 172, 17.56, 324]
[433.34, 4, 433.35, 7, 15.2, 3543, 15.21, 3967, 8.83, 215, 8.83, 139, 117.56, 5, 117.57, 11, 122.2, 20, 122.22, 4, 17.55, 172, 17.56, 324]
[433.34, 4, 433.3

[433.49, 29, 433.5, 146, 15.2, 855, 15.21, 4537, 8.82, 2561, 8.83, 2676, 117.66, 10, 117.69, 32, 122.27, 59, 122.3, 37, 17.53, 262, 17.54, 206]
[433.49, 29, 433.5, 146, 15.2, 855, 15.21, 4537, 8.82, 2573, 8.83, 2636, 117.66, 10, 117.69, 32, 122.27, 59, 122.3, 37, 17.53, 262, 17.54, 206]
[433.49, 29, 433.5, 146, 15.2, 855, 15.21, 4537, 8.82, 2573, 8.83, 2636, 117.66, 10, 117.69, 32, 122.28, 4, 122.29, 7, 17.53, 272, 17.54, 256]
[433.49, 29, 433.5, 146, 15.2, 855, 15.21, 4537, 8.82, 2573, 8.83, 2636, 117.67, 16, 117.68, 3, 122.28, 4, 122.29, 7, 17.53, 272, 17.54, 256]
[433.49, 29, 433.5, 146, 15.2, 972, 15.21, 4822, 8.82, 2573, 8.83, 2636, 117.67, 16, 117.68, 3, 122.28, 4, 122.29, 7, 17.53, 272, 17.54, 256]
[433.48, 30, 433.5, 184, 15.2, 972, 15.21, 4822, 8.82, 2573, 8.83, 2636, 117.67, 16, 117.68, 3, 122.28, 4, 122.29, 7, 17.53, 272, 17.54, 256]
[433.48, 30, 433.5, 184, 15.2, 1542, 15.21, 4537, 8.82, 2663, 8.83, 2462, 117.67, 16, 117.68, 3, 122.26, 15, 122.28, 5, 17.53, 302, 17.54, 144]

[433.48, 11, 433.49, 2, 15.19, 5437, 15.2, 1690, 8.82, 1397, 8.83, 2307, 117.67, 2, 117.68, 5, 122.28, 3, 122.3, 6, 17.53, 220, 17.54, 215]
[433.48, 8, 433.5, 119, 15.19, 5437, 15.2, 2850, 8.82, 1397, 8.83, 2307, 117.67, 14, 117.69, 16, 122.28, 3, 122.3, 6, 17.53, 220, 17.54, 215]
[433.48, 8, 433.5, 119, 15.19, 5437, 15.2, 2850, 8.82, 1397, 8.83, 2307, 117.67, 14, 117.69, 16, 122.28, 3, 122.3, 5, 17.53, 220, 17.54, 215]
[433.49, 2, 433.5, 160, 15.19, 5437, 15.2, 2850, 8.82, 1397, 8.83, 2307, 117.67, 14, 117.69, 16, 122.28, 3, 122.3, 5, 17.53, 220, 17.54, 215]
[433.49, 2, 433.5, 160, 15.19, 5437, 15.2, 2850, 8.82, 1397, 8.83, 2307, 117.67, 9, 117.69, 31, 122.28, 3, 122.3, 5, 17.53, 220, 17.54, 215]
[433.49, 2, 433.5, 159, 15.19, 5437, 15.2, 2850, 8.82, 1397, 8.83, 2307, 117.67, 9, 117.69, 31, 122.28, 3, 122.3, 5, 17.53, 220, 17.54, 215]
[433.47, 4, 433.49, 43, 15.19, 6901, 15.2, 2565, 8.82, 2528, 8.83, 2150, 117.65, 6, 117.67, 10, 122.26, 47, 122.28, 2, 17.53, 280, 17.54, 144]
[433.47, 

...

[434.34, 185, 434.35, 34, 15.17, 1881, 15.18, 6971, 8.79, 2148, 8.8, 3018, 118.37, 4, 118.4, 24, 122.77, 70, 122.79, 12, 17.42, 299, 17.43, 412]
[434.34, 185, 434.35, 34, 15.17, 270, 15.17, 421, 8.79, 2148, 8.8, 3018, 118.37, 4, 118.4, 24, 122.78, 12, 122.79, 12, 17.42, 392, 17.43, 505]
[434.34, 185, 434.35, 34, 15.17, 270, 15.17, 421, 8.79, 2148, 8.8, 3079, 118.37, 4, 118.4, 24, 122.78, 12, 122.79, 12, 17.42, 392, 17.43, 505]
[434.35, 36, 434.36, 5, 15.17, 270, 15.17, 421, 8.79, 2148, 8.8, 3079, 118.38, 8, 118.4, 24, 122.78, 12, 122.79, 12, 17.42, 392, 17.43, 505]
[434.35, 36, 434.36, 5, 15.17, 270, 15.17, 421, 8.79, 2148, 8.8, 3079, 118.38, 8, 118.4, 24, 122.78, 12, 122.79, 12, 17.42, 388, 17.43, 496]
[434.35, 46, 434.36, 5, 15.17, 270, 15.17, 421, 8.79, 2148, 8.8, 3079, 118.38, 8, 118.4, 24, 122.78, 12, 122.79, 12, 17.42, 388, 17.43, 496]
[434.35, 46, 434.36, 5, 15.17, 270, 15.17, 421, 8.79, 2148, 8.8, 3079, 118.38, 8, 118.4, 31, 122.78, 12, 122.79, 15, 17.42, 419, 17.43, 496]
[434.

[434.4, 60, 434.44, 59, 15.16, 4059, 15.17, 3507, 8.78, 2981, 8.79, 2445, 118.43, 2, 118.44, 6, 122.81, 6, 122.84, 4, 17.42, 10, 17.43, 323]
[434.4, 60, 434.44, 59, 15.16, 4059, 15.17, 3507, 8.78, 2981, 8.79, 2445, 118.42, 2, 118.44, 21, 122.81, 6, 122.84, 4, 17.42, 10, 17.43, 323]
[434.4, 60, 434.44, 34, 15.16, 4059, 15.17, 3507, 8.78, 2981, 8.79, 2445, 118.42, 2, 118.44, 21, 122.81, 6, 122.84, 4, 17.42, 10, 17.43, 323]
[434.4, 60, 434.44, 34, 15.16, 4059, 15.17, 3507, 8.78, 2981, 8.79, 2445, 118.42, 2, 118.44, 21, 122.81, 6, 122.83, 13, 17.42, 10, 17.43, 323]
[434.4, 60, 434.44, 34, 15.16, 4059, 15.17, 3507, 8.78, 2981, 8.79, 2445, 118.41, 4, 118.44, 21, 122.81, 6, 122.83, 13, 17.42, 10, 17.43, 230]
[434.39, 84, 434.43, 12, 15.16, 4059, 15.17, 3399, 8.78, 2981, 8.79, 2445, 118.41, 4, 118.44, 21, 122.8, 35, 122.83, 13, 17.42, 10, 17.43, 230]
[434.39, 84, 434.41, 3, 15.16, 4059, 15.17, 3399, 8.78, 2981, 8.79, 2445, 118.41, 4, 118.44, 21, 122.8, 35, 122.83, 31, 17.42, 690, 17.43, 323]
[

[434.38, 2, 434.4, 4, 15.16, 5229, 15.17, 1940, 8.78, 4492, 8.79, 1679, 118.39, 3, 118.42, 2, 122.79, 4, 122.81, 2, 17.42, 198, 17.43, 28]
[434.38, 2, 434.4, 5, 15.16, 5229, 15.17, 1940, 8.78, 4492, 8.79, 1679, 118.39, 3, 118.42, 2, 122.79, 4, 122.81, 2, 17.42, 198, 17.43, 28]
[434.37, 3, 434.4, 5, 15.16, 5229, 15.17, 1940, 8.78, 4492, 8.79, 1679, 118.39, 3, 118.42, 2, 122.79, 4, 122.81, 2, 17.42, 198, 17.43, 28]
[434.37, 3, 434.4, 5, 15.16, 5229, 15.17, 1940, 8.78, 4492, 8.79, 1526, 118.39, 3, 118.42, 2, 122.79, 4, 122.81, 2, 17.42, 198, 17.43, 28]
[434.37, 3, 434.4, 5, 15.16, 5229, 15.17, 1940, 8.78, 4492, 8.79, 1526, 118.39, 3, 118.42, 2, 122.79, 4, 122.81, 6, 17.42, 214, 17.43, 28]
[434.36, 110, 434.39, 8, 15.16, 5229, 15.17, 1940, 8.78, 4492, 8.79, 1526, 118.39, 3, 118.41, 2, 122.79, 4, 122.81, 6, 17.42, 214, 17.43, 28]
[434.36, 81, 434.39, 8, 15.16, 5229, 15.17, 1940, 8.78, 4492, 8.79, 1526, 118.39, 3, 118.41, 2, 122.79, 4, 122.81, 6, 17.42, 214, 17.43, 28]
[434.36, 106, 434.39, 

[434.41, 133, 434.45, 12, 15.16, 5229, 15.17, 2267, 8.78, 4507, 8.79, 1782, 118.44, 4, 118.46, 21, 122.82, 2, 122.84, 4, 17.42, 88, 17.43, 163]
[434.41, 133, 434.44, 1, 15.16, 5229, 15.17, 2267, 8.78, 4507, 8.79, 1782, 118.44, 4, 118.46, 21, 122.82, 2, 122.84, 4, 17.42, 88, 17.43, 163]
[434.41, 133, 434.44, 1, 15.16, 5229, 15.17, 2124, 8.78, 4507, 8.79, 1721, 118.44, 4, 118.46, 21, 122.81, 6, 122.84, 4, 17.42, 88, 17.43, 152]
[434.45, 3, 434.46, 115, 15.16, 5229, 15.17, 2124, 8.78, 4507, 8.79, 1721, 118.44, 4, 118.46, 21, 122.81, 6, 122.84, 4, 17.42, 88, 17.43, 152]
[434.43, 4, 434.46, 115, 15.16, 5229, 15.17, 2124, 8.78, 4507, 8.79, 1721, 118.44, 4, 118.46, 21, 122.81, 6, 122.84, 4, 17.42, 88, 17.43, 152]
[434.43, 4, 434.45, 2, 15.16, 5229, 15.17, 2124, 8.78, 4507, 8.79, 1721, 118.44, 4, 118.46, 21, 122.81, 6, 122.84, 4, 17.42, 88, 17.43, 152]
[434.43, 4, 434.45, 2, 15.16, 5229, 15.17, 2124, 8.78, 4507, 8.79, 1721, 118.44, 4, 118.46, 21, 122.81, 6, 122.84, 4, 17.42, 89, 17.43, 152]
[4

[434.5, 5, 434.53, 123, 15.16, 2192, 15.17, 2339, 8.78, 1725, 8.79, 1998, 118.49, 7, 118.52, 9, 122.86, 17, 122.88, 9, 17.41, 91, 17.42, 172]
[434.51, 2, 434.53, 78, 15.16, 2192, 15.17, 2339, 8.78, 1725, 8.79, 1998, 118.49, 7, 118.52, 9, 122.86, 17, 122.88, 9, 17.41, 91, 17.42, 172]
[434.52, 1, 434.53, 11, 15.16, 1969, 15.17, 2339, 8.78, 1602, 8.79, 1998, 118.51, 10, 118.53, 3, 122.87, 10, 122.89, 4, 17.41, 91, 17.42, 172]
[434.52, 1, 434.53, 11, 15.16, 1969, 15.17, 2339, 8.78, 1602, 8.79, 1998, 118.51, 10, 118.53, 3, 122.87, 10, 122.89, 4, 17.41, 87, 17.42, 202]
[434.52, 1, 434.53, 10, 15.16, 1969, 15.17, 2339, 8.78, 1602, 8.79, 1998, 118.51, 10, 118.53, 3, 122.87, 10, 122.89, 4, 17.41, 77, 17.42, 202]
[434.52, 1, 434.53, 10, 15.16, 1969, 15.17, 2339, 8.78, 1602, 8.79, 1998, 118.51, 10, 118.53, 3, 122.87, 10, 122.88, 2, 17.41, 77, 17.42, 202]
[434.52, 1, 434.53, 10, 15.16, 1969, 15.17, 2339, 8.78, 1602, 8.79, 1998, 118.51, 4, 118.53, 3, 122.87, 4, 122.88, 2, 17.41, 77, 17.42, 182]
[43

In [ ]:
df2 = read_multiple_prices(TICKERS, 450)
df2.to_csv('six_asset_data_0707_2.csv')

In [14]:
ib.disconnect()

## Process data

In [165]:
def calculate_mid_imb(df, num_asset):
    for i in range(1, num_asset + 1):
        df["mid{}".format(i)] = (df["bid{}".format(i)] + df["ask{}".format(i)]) / 2
        df["imb{}".format(i)] = df["bid_size{}".format(i)] / (df["bid_size{}".format(i)] + df["ask_size{}".format(i)])
    
    return df

In [168]:
df = df.sort_values(by=['time'])
df['time'] = pd.to_datetime(df['time']).dt.tz_localize(None)  # Get rid of timezone so that datetime can be compared

start_time = datetime(2021, 7, 7, 9, 30, 0)
end_time = datetime(2021, 7, 7, 16, 0, 0)

df = df.loc[(df["time"] >= start_time) & (df["time"] <= end_time)]
df = calculate_mid_imb(df, len(TICKERS))
df.to_csv("cleaned_csv/0707_new.csv", index=False)

In [170]:
df.head()

,time,bid1,bid_size1,ask1,ask_size1,bid2,bid_size2,ask2,ask_size2,bid3,...,mid2,imb2,mid3,imb3,mid4,imb4,mid5,imb5,mid6,imb6
101,2021-07-07 09:30:05.033870,433.35,20,433.37,4,15.2,3349,15.21,3459,8.83,...,15.205,0.491921,8.83,0.000691,117.585,0.058824,122.215,0.777778,17.55,0.47619
102,2021-07-07 09:30:05.033870,433.35,20,433.37,4,15.2,3534,15.21,4309,8.83,...,15.205,0.450593,8.83,0.000691,117.585,0.090909,122.215,0.777778,17.55,0.47619
103,2021-07-07 09:30:08.040008,433.35,186,433.38,18,15.2,3534,15.21,4309,8.83,...,15.205,0.450593,8.83,0.000691,117.585,0.090909,122.215,0.777778,17.55,0.47619
104,2021-07-07 09:30:15.087005,433.35,186,433.38,18,15.2,3534,15.21,4309,8.83,...,15.205,0.450593,8.83,0.000638,117.590,0.093023,122.215,0.777778,17.55,0.47619
105,2021-07-07 09:30:15.087005,433.35,186,433.38,18,15.2,3534,15.21,4309,8.83,...,15.205,0.450593,8.83,0.000638,117.590,0.093023,122.220,0.533333,17.55,0.47619


In [69]:
df_0624 = pd.read_csv("six_asset_data_0624_1.csv", index_col=0)
df_0624["time"] = df_0624["time"].apply(pd.to_datetime)
df_0624.sort_values(by=['time'])
df_0624.time -= timedelta(hours=12)
df_0624 = calculate_mid_imb(df_0624, len(TICKERS))

In [70]:
df_0624.to_csv("cleaned_csv/0624", index=False)

In [71]:
df_0623_1 = pd.read_csv("six_asset_data11.csv", index_col=0)
df_0623_1["time"] = df_0623_1["time"].apply(pd.to_datetime)
df_0623_1.sort_values(by=['time'])
df_0623_1.time -= timedelta(hours=12)
df_0623_1 = calculate_mid_imb(df_0623_1, len(TICKERS))

df_0623_2 = pd.read_csv("six_asset_data_22.csv", index_col=0)
df_0623_2["time"] = df_0623_2["time"].apply(pd.to_datetime)
df_0623_2.sort_values(by=['time'])
df_0623_2.time -= timedelta(hours=12)
df_0623_2 = calculate_mid_imb(df_0623_1, len(TICKERS))

In [72]:
df = pd.concat([df_0623_1, df_0623_2])
df.to_csv("cleaned_csv/0623", index=False)

### Aggregate a pd dataframe from all pickle files

In [171]:
all_pkl_files = os.listdir("data")

In [194]:
all_pkl_files = [file for file in all_pkl_files if file.startswith("1625")]

In [195]:
# Aggregate .pkl data
# all_pkl_files = os.listdir("data")
all_data = []

columns=['time']
for i in range(1, 7):
    columns.extend(['bid{}'.format(i), 'bid_size{}'.format(i), 'ask{}'.format(i), 'ask_size{}'.format(i)])

for file in all_pkl_files:
    temp = pickle.load(open( os.path.join("data", file), "rb" ))
    all_data.extend(temp)
    
df = pd.DataFrame(all_data, columns=columns)

In [196]:
df = df.sort_values(by=['time'])
df['time'] = pd.to_datetime(df['time']).dt.tz_localize(None)  # Get rid of timezone so that datetime can be compared

start_time = datetime(2021, 7, 6, 9, 30, 0)
end_time = datetime(2021, 7, 6, 16, 0, 0)

df = df.loc[(df["time"] >= start_time) & (df["time"] <= end_time)]
df = calculate_mid_imb(df, len(TICKERS))
df = df.reset_index(drop=True)
df.to_csv("cleaned_csv/0706_new.csv", index=False)

In [141]:
# df_pair_1 = df_from_pickle 
df_pair_1 = df_from_pickle.filter(regex='1|2|time')
df_pair_1["mid1"] = 0.5 * (df_pair_1["bid1"] + df_pair_1["ask1"])
df_pair_1["mid2"] = 0.5 * (df_pair_1["bid2"] + df_pair_1["ask2"])

<ipython-input-141-558df35a0132>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_pair_1["mid1"] = 0.5 * (df_pair_1["bid1"] + df_pair_1["ask1"])
<ipython-input-141-558df35a0132>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_pair_1["mid2"] = 0.5 * (df_pair_1["bid2"] + df_pair_1["ask2"])


In [142]:
# df_pair_1.time += timedelta(hours=5)
df_pair_1 = df_pair_1.sort_values(by=['time'])
df_pair_1['time'] = pd.to_datetime(df_pair_1['time']).dt.tz_localize(None)  # Get rid of timezone so that datetime can be compared

start_time = datetime(2021, 7, 6, 9, 30, 0)
end_time = datetime(2021, 7, 6, 16, 0, 0)

df_pair_1.loc[(df_pair_1["time"] >= start_time) & (df_pair_1["time"] <= end_time)]
df_pair_1.to_csv("cleaned_csv/0706.csv", index=False)

In [152]:
df_pair_1.head()

,time,bid1,bid_size1,ask1,ask_size1,bid2,bid_size2,ask2,ask_size2,mid1,mid2
3202,2021-07-06 09:24:59.172586,433.55,5,433.58,39,15.19,3339,15.2,3315,433.565,15.195
3203,2021-07-06 09:24:59.172586,433.55,5,433.58,39,15.19,3339,15.2,3315,433.565,15.195
3204,2021-07-06 09:25:09.168021,433.56,10,433.58,2,15.19,3339,15.2,3387,433.570,15.195
3205,2021-07-06 09:25:18.216665,433.54,4,433.56,3,15.19,3509,15.2,3028,433.550,15.195
3206,2021-07-06 09:25:18.216665,433.54,4,433.56,3,15.19,3679,15.2,3315,433.550,15.195


In [147]:
df = pd.read_csv("cleaned_csv/0624")
start_time = datetime(2021, 6, 25, 9, 30, 0)
end_time = datetime(2021, 6, 25, 16, 0, 0)

df["time"] = pd.to_datetime(df["time"]).dt.tz_localize(None)
df = df.loc[(df["time"] >= start_time) & (df["time"] <= end_time)].drop_duplicates()
df.to_csv("cleaned_csv/0625_new.csv", index=False)

In [153]:
df.head()

,time,bid1,bid_size1,ask1,ask_size1,bid2,bid_size2,ask2,ask_size2,bid3,...,mid2,imb2,mid3,imb3,mid4,imb4,mid5,imb5,mid6,imb6
693,2021-06-25 09:30:09.402665,425.95,27,425.97,85,15.47,3220,15.48,4154,9.14,...,15.475,0.436669,9.145,0.495201,111.720,0.523810,118.140,0.098361,18.505,0.348760
694,2021-06-25 09:30:10.323049,425.95,27,425.97,85,15.47,3220,15.48,4154,9.14,...,15.475,0.436669,9.145,0.495201,111.720,0.523810,118.140,0.098361,18.505,0.349338
695,2021-06-25 09:30:19.334121,425.96,16,425.98,160,15.47,3220,15.48,4154,9.14,...,15.475,0.436669,9.145,0.495201,111.720,0.523810,118.140,0.411765,18.505,0.349338
696,2021-06-25 09:30:25.273061,425.96,16,425.98,160,15.47,3220,15.48,4154,9.14,...,15.475,0.436669,9.145,0.495201,111.725,0.017857,118.140,0.411765,18.505,0.355219
697,2021-06-25 09:30:29.199610,425.95,1,425.97,59,15.47,3150,15.48,4154,9.14,...,15.475,0.431271,9.145,0.474874,111.725,0.017857,118.135,0.238095,18.505,0.355219


Peer closed connection.


In [ ]:
def plot(y_ticker, x_ticker, period=None, price_type="Close"):
    """ 
    Usage: To do a linear regression of y=SPY on x=SH, call `plot("SPY", "SH")`
    This function plots two graph about the relationship between two tickers, and do OLS with them.
        plot 1: The scatter plot of two etf prices
        plot 2: The lineplot of two etf prices over time
    Arguments:
        y_ticker: ticker of the etf to be the dependent variable
        x_ticker: ticker of the etf to be the independent variable
        period (day): If given, use the last `period` number of closed price 
                    If not given, use all the data since both tickers are traded
    """
    
    y = data[y_ticker][price_type] if y_ticker in data else collect_historical(y_ticker)
    x = data[x_ticker][price_type] if x_ticker in data else collect_historical(x_ticker)
    
    period = min(len(y), len(x)) if period == None else period
    y = y[-period:]
    x = x[-period:]
    
    data_pair = pd.DataFrame()
    data_pair[x_ticker], data_pair[y_ticker], data_pair["Year"] = x, y, list(map(lambda x: str(x.year), x.index))
    
    plt.figure(figsize=(20,8))
    plt.subplot(1,2,1)
    plt.title("Relationship between {} and {}".format(y_ticker, x_ticker), fontsize=20)
    sns.scatterplot(data=data_pair, x=x_ticker, y=y_ticker, hue="Year")
    
    x = sm.add_constant(x)
    model = sm.OLS(y, x)
    result = model.fit()
    constant = result.params[0]
    slope = result.params[1]
    
    plt.text((max(x["Close"]) + min(x["Close"])) / 2, 0.95*max(y), r'${}={}*{} + {}$'.format(
        y_ticker, round(slope, 2), x_ticker, round(constant, 2)), fontsize=20, ha='center')
    
    plt.subplot(1,2,2)
    plt.title("Price Movement of {} and {}".format(y_ticker, x_ticker), fontsize=20)
    sns.lineplot(data=data_pair)
    plt.ylabel("Price")
    plt.show()
    plt.close()
    
    return data_pair, slope, constant

In [ ]:
def calculate_parameters(df):  # calculate in-sample beta, constant
    df['mid1']=(df.bid1+df.ask1)/2
    df['mid2']=(df.bid2+df.ask2)/2
    
    X = sm.add_constant(df.mid2)
    res = sm.OLS(df.mid1, X).fit()
    constant=res.params[0]
    slope=res.params[1]
    
    predicted_Y=constant+slope*df.mid2
    df['residuals']=df.mid1-predicted_Y
    
    return slope, constant


def prep_data(df, slope, constant, res_num, imb1_num, imb2_num):  # symmetrize data and cut states  
    df_flip=df.copy()
    df_flip.columns=['time', 'ask1', 'ask_size1', 'bid1', 'bid_size1', 'ask2', 'ask_size2', 'bid2', 'bid_size2',
                     'mid1','mid2','residuals']
    df_flip[['ask1', 'bid1', 'ask2', 'bid2','residuals']]=-df_flip[['ask1', 'bid1', 'ask2', 'bid2','residuals']]

    change1=df.bid1[len(df)-1]-df_flip.bid1[0]-0.01
    change2=df.bid2[len(df)-1]-df_flip.bid2[0]-0.01

    df_flip[['ask1', 'bid1']]=df_flip[['ask1', 'bid1']]+change1
    df_flip[['ask2', 'bid2']]=df_flip[['ask2', 'bid2']]+change2
    
    df_flip['mid1']=(df_flip.bid1+df_flip.ask1)/2
    df_flip['mid2']=(df_flip.bid2+df_flip.ask2)/2
    predicted_Y_flip=constant+slope*df_flip.mid2

    df_flip.time=pd.to_datetime(df_flip.time)
    df_flip.time+=timedelta(hours=5)
    
    df.set_index("time",inplace=True)
    df.index = pd.to_datetime(df.index,utc=True)
    df['imb1']=df.bid_size1/(df.bid_size1+df.ask_size1)
    df['imb2']=df.bid_size2/(df.bid_size2+df.ask_size2)
    df2=df[['residuals','mid1','mid2','imb1','imb2']]
    df2.index = df.index.shift(-10,freq='S')
    df2.columns=['residual_later','mid1_later','mid2_later','imb1_later','imb2_later']
    df = pd.merge_asof(df, df2, left_index=True, right_index=True, direction='forward')
    df['pnl'] = df.residual_later-df.residuals  # forward pnl
    df['mid1_diff'] = df.mid1_later-df.mid1
    df['mid2_diff'] = df.mid2_later-df.mid2
    df.dropna(inplace=True)
    
    df_flip.set_index("time",inplace=True)
    df_flip.index = pd.to_datetime(df_flip.index,utc=True)
    df_flip['imb1']=df_flip.bid_size1/(df_flip.bid_size1+df_flip.ask_size1)
    df_flip['imb2']=df_flip.bid_size2/(df_flip.bid_size2+df_flip.ask_size2)
    df2_flip=df_flip[['residuals','mid1','mid2','imb1','imb2']]
    df2_flip.index = df_flip.index.shift(-10,freq='S')# - timedelta(minutes=5)  # data from 5 minutes later
    df2_flip.columns=['residual_later','mid1_later','mid2_later','imb1_later','imb2_later']
    df_flip = pd.merge_asof(df_flip, df2_flip, left_index=True, right_index=True, direction='forward')
    df_flip['pnl'] = df_flip.residual_later-df_flip.residuals  # forward pnl
    df_flip['mid1_diff'] = df_flip.mid1_later-df_flip.mid1
    df_flip['mid2_diff'] = df_flip.mid2_later-df_flip.mid2
    df_flip.dropna(inplace=True)
    
    df=pd.concat([df,df_flip])

    df.index = pd.to_datetime(df.index,utc=True)
    
    df['residual_bucket'] = pd.cut(df['residuals'], res_num, labels=False)
    
    df['imb1']=df.bid_size1/(df.bid_size1+df.ask_size1)
    df['imb2']=df.bid_size2/(df.bid_size2+df.ask_size2)
    df['imb1_bucket'] = pd.cut(df.imb1, imb1_num, labels=False)
    df['imb2_bucket'] = pd.cut(df.imb2, imb2_num, labels=False)
    return df

In [ ]:
slope, constant = calculate_parameters(df)
df = prep_data(df, slope, constant, res_num, imb1_num, imb2_num)